In [1]:
from pathlib import Path
import pandas as pd
import requests
import os

In [2]:
home_dir = Path.home()
inside_airbnb_data_dir = home_dir / 'Programming/data/inside-airbnb/london'
population_dataset_england_wales_data_dir = home_dir / 'Programming/data/office-national-statistics/population-dataset-england-wales/'
crime_rate_dir = home_dir / 'Programming/data/crime-rate/'

In [3]:
FOURSQUARE_API_KEY = os.environ['FOURSQUARE_API_KEY']
FOURSQUARE_URL = "https://api.foursquare.com/v3/places/search"

In [5]:
inside_airbnb_data_file = inside_airbnb_data_dir / 'listings.csv'
population_dataset_england_wales_data_file = population_dataset_england_wales_data_dir / 'mye23tablesew.xlsx'
crime_rate_data_file = crime_rate_dir / 'crimerate-pro-data-table-rmp-region-towns-cities.csv'

In [6]:
crime_rate_df = pd.read_csv(crime_rate_data_file, usecols=['Borough', 'Crime Rate'])
crime_rate_df.rename(columns={'Borough': 'borough', 'Crime Rate': 'crime_rate'}, inplace=True)
crime_rate_df = crime_rate_df[crime_rate_df.borough != 'DownloadCSVExcelTSV']

In [7]:
columns_list = ['neighbourhood_cleansed', 'bathrooms', 'bedrooms', 'latitude', 'longitude',
                'room_type', 'latitude', 'longitude', 'property_type', 'price']
inside_airbnb_df = pd.read_csv(inside_airbnb_data_file, usecols=columns_list)
inside_airbnb_df.rename(columns={'neighbourhood_cleansed': 'borough'}, inplace=True)
inside_airbnb_df.price = inside_airbnb_df.price.str.replace('$', '')

In [8]:
inside_airbnb_df = inside_airbnb_df.merge(crime_rate_df, on='borough', how='left')
inside_airbnb_df.head()

,borough,latitude,longitude,property_type,room_type,bathrooms,bedrooms,price,crime_rate
0,Hammersmith and Fulham,51.49392,-0.22754,Entire rental unit,Entire home/apt,1.5,1.0,89.00,103.14
1,Tower Hamlets,51.52435,-0.06938,Entire condo,Entire home/apt,3.0,3.0,220.00,98.60
2,Hammersmith and Fulham,51.49547,-0.22864,Entire rental unit,Entire home/apt,1.0,1.0,88.00,103.14
3,Hammersmith and Fulham,51.49368,-0.22774,Room in aparthotel,Private room,1.0,1.0,75.00,103.14
4,Haringey,51.59031,-0.09408,Private room in rental unit,Private room,1.0,1.0,53.00,133.11


In [42]:
inside_airbnb_df.shape[0]*(0.0066)

627.9504

In [37]:
CATEGORY_MAPPING = {
    # Restaurants
    "Italian Restaurant": "Restaurant",
    "Pizzeria": "Restaurant",
    "French Restaurant": "Restaurant",
    "Japanese Restaurant": "Restaurant",
    "Chinese Restaurant": "Restaurant",
    "Indian Restaurant": "Restaurant",
    "Turkish Restaurant": "Restaurant",
    "Dim Sum Restaurant": "Restaurant",
    "Mexican Restaurant": "Restaurant",
    "Thai Restaurant": "Restaurant",
    "Brazilian Restaurant": "Restaurant",
    "Caribbean Restaurant": "Restaurant",
    "Vietnamese Restaurant": "Restaurant",
    "Ethiopian Restaurant": "Restaurant",
    "Korean Restaurant": "Restaurant",
    "Mediterranean Restaurant": "Restaurant",
    "Greek Restaurant": "Restaurant",
    "American Restaurant": "Restaurant",
    "BBQ Joint": "Restaurant",
    "Burger Joint": "Restaurant",
    "Fast Food Restaurant": "Restaurant",
    "Steakhouse": "Restaurant",
    "Seafood Restaurant": "Restaurant",
    "Sushi Restaurant": "Restaurant",
    "Vegetarian / Vegan Restaurant": "Restaurant",
    "Fish and Chips Shop": "Restaurant",

    # Cafés & Bakeries
    "Coffee Shop": "Cafe",
    "Café": "Cafe",
    "Bakery": "Cafe",
    "Bagel Shop": "Cafe",
    "Tea House": "Cafe",
    "Ice Cream Shop": "Cafe",
    "Dessert Shop": "Cafe",

    # Bars & Nightlife
    "Bar": "Nightlife",
    "Pub": "Nightlife",
    "Cocktail Bar": "Nightlife",
    "Wine Bar": "Nightlife",
    "Lounge": "Nightlife",
    "Nightclub": "Nightlife",
    
    # Shopping & Retail
    "Shopping Mall": "Retail",
    "Clothing Store": "Retail",
    "Shoe Store": "Retail",
    "Electronics Store": "Retail",
    "Bookstore": "Retail",
    "Furniture Store": "Retail",
    "Jewelry Store": "Retail",
    "Toy Store": "Retail",
    "Supermarket": "Grocery Store",
    "Convenience Store": "Grocery Store",

    # Fitness & Wellness
    "Gym": "Fitness",
    "Yoga Studio": "Fitness",
    "Pilates Studio": "Fitness",
    "CrossFit Gym": "Fitness",
    "Martial Arts Studio": "Fitness",
    "Cycling Studio": "Fitness",
    "Spa": "Wellness",
    "Massage Therapy": "Wellness",
    "Sauna": "Wellness",

    # Entertainment & Cultural
    "Movie Theater": "Entertainment",
    "Theater": "Entertainment",
    "Concert Hall": "Entertainment",
    "Comedy Club": "Entertainment",
    "Escape Room": "Entertainment",
    "Arts and Entertainment": "Entertainment",
    "Museum": "Cultural",
    "Art Gallery": "Cultural",
    "Library": "Cultural",
    "Historical Site": "Cultural",

    # Parks & Outdoor Activities
    "Park": "Outdoor",
    "Botanical Garden": "Outdoor",
    "Hiking Trail": "Outdoor",
    "Nature Reserve": "Outdoor",
    "Zoo": "Outdoor",
    "Aquarium": "Outdoor",
    "Beach": "Outdoor",
    
    # Transportation
    "Train Station": "Transport",
    "Bus Station": "Transport",
    "Subway Station": "Transport",
    "Taxi Stand": "Transport",
    "Car Rental": "Transport",
    "Parking Lot": "Transport",
    
    # Services & Miscellaneous
    "Bank": "Services",
    "ATM": "Services",
    "Post Office": "Services",
    "Hair Salon": "Services",
    "Barbershop": "Services",
    "Dry Cleaner": "Services",
    "Pharmacy": "Healthcare",
    "Hospital": "Healthcare",
    "Clinic": "Healthcare",
    "Mental Health Clinic": "Healthcare",
    "Alternative Medicine Clinic": "Healthcare",
    "Dentist": "Healthcare",
    "Veterinarian": "Healthcare",
    "Physical Therapy Clinic": "Healthcare",
    "Tutoring Service": "Education",
    "Nursery School": "Education",
    "Spiritual Center": "Spiritual",
}

In [17]:
def get_nearby_categories(lat, lon, radius=150, limit=3):
    headers = {
        "Authorization": FOURSQUARE_API_KEY,
        "Accept": "application/json"
    }
    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        "limit": limit
    }

    if not FOURSQUARE_API_KEY:
        return "API key missing"

    response = requests.get(FOURSQUARE_URL, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        categories = set()
        for place in data.get('results', []):
            category_list = place.get('categories', [])
            if category_list:
                category_name = category_list[0]['name']
                # Map category to broader category if found
                consolidated_category = CATEGORY_MAPPING.get(category_name, category_name)
                categories.add(consolidated_category)
        
        return ', '.join(categories) if categories else "No amenities found"
    else:
        return f"API error: {response.status_code}"

In [38]:
inside_airbnb_df.iloc[30:60].apply(lambda row: get_nearby_categories(row['latitude'], row['longitude']), axis=1)

30                            Restaurant, Entertainment
31                         Primary and Secondary School
32                                             Cultural
33                                          Print Store
34    Heating, Ventilating and Air Conditioning Cont...
35                 Restaurant, Gourmet Store, Brasserie
36                                    Church, Preschool
37                            Nightlife, Car Dealership
38                               Landmarks and Outdoors
39                          Doctor's Office, Healthcare
40                 Grocery Store, Restaurant, Nightlife
41                            Cafe, Drugstore, Services
42                             Cosmetics Store, Outdoor
43                    Grocery Store, Child Care Service
44                    Nightlife, College and University
45          Landmarks and Outdoors, Nightlife, Cultural
46                 Restaurant, Nightlife, Entertainment
47                    Construction, Monument, Ni

In [10]:
inside_airbnb_df.head()

,borough,latitude,longitude,property_type,room_type,bathrooms,bedrooms,price,crime_rate
0,Hammersmith and Fulham,51.49392,-0.22754,Entire rental unit,Entire home/apt,1.5,1.0,89.00,103.14
1,Tower Hamlets,51.52435,-0.06938,Entire condo,Entire home/apt,3.0,3.0,220.00,98.60
2,Hammersmith and Fulham,51.49547,-0.22864,Entire rental unit,Entire home/apt,1.0,1.0,88.00,103.14
3,Hammersmith and Fulham,51.49368,-0.22774,Room in aparthotel,Private room,1.0,1.0,75.00,103.14
4,Haringey,51.59031,-0.09408,Private room in rental unit,Private room,1.0,1.0,53.00,133.11
